In [1]:
! pip install transformers==4.30.2
! pip install datasets
! pip install evaluate
! pip install accelerate

In [2]:
import torch
from transformers import AutoTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from datasets import Dataset
import evaluate

In [3]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
device

device(type='cpu')

In [4]:
df_train = pd.read_csv("llm-classification-finetuning/train.csv")
df_test = pd.read_csv("llm-classification-finetuning/test.csv")

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

c:\Users\70p6229\computer_vision\examples\pytorch\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels = 3,
).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.we

In [7]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

66955779

In [8]:
df_train['text'] = 'prompt' + df_train['prompt'] + 'response_a' + df_train['response_a'] + 'response_b' + df_train['response_b']

In [9]:
df_train['label'] = np.argmax(df_train[['winner_model_a', 'winner_model_b', 'winner_tie']], axis=1)

In [10]:
df_train = df_train[['text', 'label']]

In [11]:
df_train, df_val = train_test_split(df_train, test_size=0.2)

In [12]:
train_data = Dataset.from_pandas(df_train)
val_data = Dataset.from_pandas(df_val)

In [13]:
train_data = train_data.map(lambda samples: tokenizer(samples["text"], padding="longest", truncation=True), batched=True)
train_data = train_data.remove_columns(['__index_level_0__'])
val_data = val_data.map(lambda samples: tokenizer(samples["text"], padding="longest", truncation=True), batched=True)
val_data = val_data.remove_columns(['__index_level_0__'])

Map:   0%|          | 0/45981 [00:00<?, ? examples/s]

Map:   0%|          | 0/11496 [00:00<?, ? examples/s]

In [14]:
training_args = TrainingArguments(
    output_dir="outputs",
    num_train_epochs=1,
    evaluation_strategy="epoch"
)

In [15]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Using the latest cached version of the module from C:\Users\70p6229\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Fri Apr  5 09:13:55 2024) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


In [16]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_args,
    compute_metrics=compute_metrics
)
trainer.train()

c:\Users\70p6229\computer_vision\examples\pytorch\.venv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5748 [00:00<?, ?it/s]

{'loss': 1.1027, 'learning_rate': 4.565066109951287e-05, 'epoch': 0.09}
{'loss': 1.0982, 'learning_rate': 4.130132219902575e-05, 'epoch': 0.17}
{'loss': 1.0979, 'learning_rate': 3.695198329853862e-05, 'epoch': 0.26}
{'loss': 1.1034, 'learning_rate': 3.26026443980515e-05, 'epoch': 0.35}
{'loss': 1.1092, 'learning_rate': 2.8253305497564368e-05, 'epoch': 0.43}
{'loss': 1.0946, 'learning_rate': 2.3903966597077245e-05, 'epoch': 0.52}
{'loss': 1.0964, 'learning_rate': 1.955462769659012e-05, 'epoch': 0.61}
{'loss': 1.0977, 'learning_rate': 1.5205288796102993e-05, 'epoch': 0.7}
{'loss': 1.0899, 'learning_rate': 1.0855949895615868e-05, 'epoch': 0.78}
{'loss': 1.0938, 'learning_rate': 6.506610995128741e-06, 'epoch': 0.87}
{'loss': 1.0891, 'learning_rate': 2.1572720946416146e-06, 'epoch': 0.96}


  0%|          | 0/1437 [00:00<?, ?it/s]

{'eval_loss': 1.088136076927185, 'eval_accuracy': 0.37795755045233126, 'eval_runtime': 1655.6572, 'eval_samples_per_second': 6.943, 'eval_steps_per_second': 0.868, 'epoch': 1.0}
{'train_runtime': 23099.2482, 'train_samples_per_second': 1.991, 'train_steps_per_second': 0.249, 'train_loss': 1.097205409592195, 'epoch': 1.0}


TrainOutput(global_step=5748, training_loss=1.097205409592195, metrics={'train_runtime': 23099.2482, 'train_samples_per_second': 1.991, 'train_steps_per_second': 0.249, 'train_loss': 1.097205409592195, 'epoch': 1.0})

In [17]:
def predict(data, model):
    model.eval()

    yhats = np.array([[1, 1, 1]])

    with torch.no_grad():
        for samples in data:
            outputs = model(torch.tensor(samples['input_ids'])).logits.cpu()
            outputs = torch.nn.functional.softmax(outputs, dim=1).numpy()
            yhats = np.concatenate((yhats, outputs))

    return np.delete(yhats, 0, 0)

In [18]:
yhat_train = predict(train_data, model)

In [19]:
accuracy_score(np.array(train_data["label"]), np.argmax(yhat_train, axis=1))

0.35625584480546313

In [20]:
log_loss(np.array(train_data["label"]), yhat_train)

c:\Users\70p6229\computer_vision\examples\pytorch\.venv\lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


1.0959395457759566

In [21]:
yhat_val = predict(val_data, model)

In [22]:
accuracy_score(np.array(val_data["label"]), np.argmax(yhat_val, axis=1))

0.351687543493389

In [23]:
log_loss(np.array(val_data["label"]), yhat_val)

c:\Users\70p6229\computer_vision\examples\pytorch\.venv\lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


1.099352674351887